In [1]:
import sys

sys.path.append("/hpc/home/ps325/projects/velorama/velorama")

import gaston_dynamics as gd
import scanpy as sc
import numpy as np 
import pandas as pd


In [2]:
import os
from collections import defaultdict
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
from glmpca import glmpca
from itertools import combinations
import torch

import sys
from importlib import reload

sys.path.append("/hpc/home/ps325/projects/gaston-velorama/src/gaston-base")

import gaston


import seaborn as sns
import math

In [3]:
from gaston import neural_net,cluster_plotting, dp_related, segmented_fit
from gaston import binning_and_plotting, isodepth_scaling, run_slurm_scripts
from gaston import spatial_gene_classification, plot_cell_types, filter_genes, process_NN_output

In [4]:
import requests

# Replace this URL with the GitHub raw file URL you copied
file_url = 'https://raw.githubusercontent.com/raphael-group/copulacci/main/src/copulacci/_data/LRdatabase/CellTalkDB/lrpairs.csv'

# Replace 'downloaded_file.extension' with the desired file name
output_file = 'lrpairs.csv'

# Downloading the file
response = requests.get(file_url)

# Writing the content to a file
with open(output_file, 'wb') as file:
    file.write(response.content)

print(f"File downloaded as {output_file}")

File downloaded as lrpairs.csv


In [5]:
df = pd.read_csv('lrpairs.csv')

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]


# Filtering the DataFrame
mouse_df = df[df['species'] == 'Mouse']

# Display the filtered data
print(mouse_df)

       ligand receptor species
3398     Wnt3     Fzd6   Mouse
3399     Wnt3     Egfr   Mouse
3400     Wnt3     Fzd2   Mouse
3401     Wnt3     Lrp5   Mouse
3402     Wnt3     Fzd1   Mouse
...       ...      ...     ...
5422     Nrg4    Erbb4   Mouse
5423  Adamts3    Ccbe1   Mouse
5424     Bmp6   Acvr2b   Mouse
5425    Pdgfd   Pdgfra   Mouse
5426     Apoe    Vldlr   Mouse

[2029 rows x 3 columns]


In [6]:
CDATADIR = "gaston-base/gaston/docs/notebooks/tutorials/"
counts_mat=np.load(CDATADIR + 'cerebellum_data/cerebellum_counts_mat.npy') # N x G UMI count array
coords_mat=np.load(CDATADIR + 'cerebellum_data/cerebellum_coords_mat.npy') # N x 2 spatial coordinate matrix
gene_labels=np.load(CDATADIR + 'cerebellum_data/cerebellum_gene_labels.npy', allow_pickle=True) # list of names for G genes
print(counts_mat.shape)

(9985, 23096)


In [7]:
def column_to_txt(df, column, output_file):
    column_values = df[f'{column}'].unique().tolist()
    with open(f'{output_file}.txt', 'w') as file:
        for value in column_values:
            file.write(f"{value}\n")

    print(f"Column values saved to '{output_file}.txt'")


In [8]:
column_to_txt(df, "ligand", "ligand")
column_to_txt(df, "receptor", "receptor")

Column values saved to 'ligand.txt'
Column values saved to 'receptor.txt'


In [9]:
A=np.load(CDATADIR + 'cerebellum_data/F_glmpca_penalty_10_rep1.npy') # GLM-PCA results used in manuscript
S=coords_mat

# z-score normalize S and A
S_torch, A_torch = neural_net.load_rescale_input_data(S,A)

In [10]:
# gaston_model, A, S = gd.gaston_model('tutorial_outputs', S_torch, A_torch)
gaston_model, A, S= process_NN_output.process_files(CDATADIR + 'cerebellum_data/reproduce_cerebellum')


best model: gaston-base/gaston/docs/notebooks/tutorials/cerebellum_data/reproduce_cerebellum/seed19


In [11]:
CDATADIR = "gaston-base/gaston/docs/notebooks/tutorials/"

In [12]:
cell_type_mod_df = gd.create_cell_type_df(CDATADIR + 'cerebellum_data/cell_type_df.csv')

In [13]:
from importlib import reload
reload(gd)

<module 'gaston_dynamics' from '/hpc/home/ps325/projects/velorama/velorama/gaston_dynamics.py'>

In [14]:
gaston_isodepth, gaston_labels = gd.calculate_isodepth(gaston_model, A, S, coords_mat, num_layers=4,
    scale_factor=64/100, flip=True, adjust = True)

import warnings
warnings.filterwarnings("ignore")

ct_list= ['Oligodendrocytes', 'Granule', 'Bergmann', 'Purkinje', 'MLI1', 'MLI2']
cell_type_df=pd.read_csv(CDATADIR + 'cerebellum_data/cell_type_df.csv', index_col=0)
pw_fit_dict, binning_output = gd.binning_output(counts_mat, gene_labels, gaston_labels, gaston_isodepth, cell_type_df, ct_list)
q=0.9 
file = 'transcription_factors.txt'

df = pd.DataFrame(counts_mat, columns=gene_labels)
adata=sc.AnnData(df)
adata.obsm["coords"] = coords_mat

Poisson regression for ALL cell types


  4%|▎         | 78/2193 [00:01<00:32, 65.64it/s]


KeyboardInterrupt: 

In [ ]:
dag_adjacency_matrix = gd.construct_dag(adata, gaston_isodepth, coords_mat, n_neighbors=7, plot=False)
gd.create_lr_targets(adata, pw_fit_dict, binning_output, q=0.9, file1 = 'receptor.txt', file2 = 'ligand.txt')

In [ ]:
print(adata.var)
l = np.array(adata.var.index.tolist())
print(list(l[adata.var.is_target]))

In [ ]:
adata.write('/hpc/home/ps325/projects/velorama/datasets/lr_cerebellum_flip.h5ad')